<a href="https://colab.research.google.com/github/novice108/ITT2025/blob/main/CBF_PID_PSO_safa_Others.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# %% [markdown]
# # Оптимізована симуляція зграї агентів
# **Поліпшена версія з фіксом зіткнень та швидкодією**

# %%
!pip install cvxpy osqp numpy matplotlib pandas
import numpy as np
import cvxpy as cp
from dataclasses import dataclass, field
from typing import List
import time
import pandas as pd

# %% [markdown]
# ## 1. Оптимізовані базові класи

# %%
@dataclass
class PIDController:
    """Ефективний PID-контролер"""
    Kp: float = 0.8
    Ki: float = 0.1
    Kd: float = 0.3
    target: float = 2.0
    integral: float = 0.0
    prev_error: float = 0.0
    dt: float = 0.1

    def compute(self, current: float) -> float:
        error = self.target - current
        self.integral += error * self.dt
        derivative = (error - self.prev_error) / self.dt
        self.prev_error = error
        return self.Kp*error + self.Ki*self.integral + self.Kd*derivative

@dataclass
class Agent:
    """Оптимізована модель агента"""
    id: int
    position: np.ndarray = field(default_factory=lambda: np.zeros(2))
    velocity: np.ndarray = field(default_factory=lambda: np.zeros(2))
    pid: PIDController = field(default_factory=PIDController)
    radius: float = 1.0

    def update_velocity(self, new_velocity: np.ndarray):
        """Векторизоване оновлення швидкості"""
        speed = np.linalg.norm(new_velocity)
        if speed > 0:
            correction = self.pid.compute(speed)
            self.velocity = new_velocity * (1 + correction/speed)

# %% [markdown]
# ## 2. Поліпшені алгоритми керування

# %%
class FastBoidsController:
    """Прискорена версія Boids алгоритму"""
    def compute_control(self, agent: Agent, neighbors: List[Agent]) -> np.ndarray:
        if not neighbors:
            return np.zeros(2)

        positions = np.array([n.position for n in neighbors])
        velocities = np.array([n.velocity for n in neighbors])

        # Векторизовані обчислення
        diffs = agent.position - positions
        dists = np.linalg.norm(diffs, axis=1)

        sep_mask = dists < 3.0
        if np.any(sep_mask):
            separation = np.mean(diffs[sep_mask]/(dists[sep_mask, None]**2 + 1e-5), axis=0)
        else:
            separation = np.zeros(2)

        alignment = np.mean(velocities, axis=0)
        cohesion = np.mean(positions, axis=0) - agent.position

        return 1.5*separation + alignment + 0.5*cohesion

class EfficientCBFPIDController:
    """Оптимізований CBF-PID контролер"""
    def __init__(self):
        self.u = cp.Variable(2)
        self.prob = None

    def compute_control(self, agent: Agent, neighbors: List[Agent]) -> np.ndarray:
        if not neighbors:
            return np.zeros(2)

        constraints = [
            cp.sum_squares(agent.position - n.position) >= (2*agent.radius)**2
            for n in neighbors
        ]

        self.prob = cp.Problem(
            cp.Minimize(cp.sum_squares(self.u)),
            constraints
        )
        self.prob.solve(solver=cp.OSQP, verbose=False)
        return self.u.value if self.u.value is not None else np.zeros(2)

# %% [markdown]
# ## 3. Оптимізована симуляція

# %%
def run_simulation(agents: List[Agent], controller, steps: int = 100):
    """Прискорена версія симуляції"""
    collisions = 0
    start_time = time.time()

    for _ in range(steps):
        # Попередньо обчислюємо позиції для оптимізації
        positions = np.array([a.position for a in agents])

        for i, agent in enumerate(agents):
            # Швидкий пошук сусідів
            dists = np.linalg.norm(positions - agent.position, axis=1)
            neighbor_indices = np.where((dists > 0) & (dists < 5.0))[0]
            neighbors = [agents[j] for j in neighbor_indices]

            # Обчислення керування
            control = controller.compute_control(agent, neighbors)
            agent.update_velocity(control)
            agent.position += agent.velocity * agent.pid.dt

        # Векторизована перевірка зіткнень
        dist_matrix = np.linalg.norm(positions[:, None] - positions, axis=2)
        np.fill_diagonal(dist_matrix, np.inf)
        collisions += np.sum(dist_matrix < 2*agents[0].radius) // 2

    return {
        "time": time.time() - start_time,
        "collisions": collisions,
        "avg_speed": np.mean([np.linalg.norm(a.velocity) for a in agents])
    }

# %% [markdown]
# ## 4. Покращені тести

# %%
def test_straight_line_motion():
    """Оптимізований тест"""
    agents = [Agent(id=i, position=np.array([i*4.0, 0.])) for i in range(20)]  # Збільшений початковий інтервал

    print("=== Удосконалений тест прямолінійного руху ===")
    results = []

    controllers = [FastBoidsController(), EfficientCBFPIDController()]

    for controller in controllers:
        # Копіюємо агенти для кожного контролера
        agents_copy = [
            Agent(id=a.id, position=a.position.copy(),
                  velocity=a.velocity.copy(), radius=a.radius)
            for a in agents
        ]

        res = run_simulation(agents_copy, controller, steps=50)
        results.append({
            "Algorithm": controller.__class__.__name__,
            "Collisions": res["collisions"],
            "Avg Speed": f"{res['avg_speed']:.2f} m/s",
            "Time": f"{res['time']:.2f} sec"
        })

    print(pd.DataFrame(results))

# %% [markdown]
# ## 5. Запуск

# %%
if __name__ == "__main__":
    test_straight_line_motion()

In [ ]:
# %% [markdown]
# # Фінальна версія симуляції зграї агентів
# **З повною реалізацією всіх алгоритмів**

# %%
!pip install cvxpy osqp numpy matplotlib pandas
import numpy as np
import cvxpy as cp
from dataclasses import dataclass, field
from typing import List, Optional
import time
import pandas as pd
import random

# %% [markdown]
# ## 1. Фінальні класи агентів

# %%
@dataclass
class PIDController:
    Kp: float = 0.8
    Ki: float = 0.1
    Kd: float = 0.3
    target: float = 2.0
    integral: float = 0.0
    prev_error: float = 0.0
    dt: float = 0.1
    safety_zone: str = "normal"

    @property
    def safety_penalty(self) -> float:
        """Новий метод для обчислення штрафу зони безпеки"""
        return {
            "critical": 0.9,
            "warning": 0.5,
            "normal": 0.1
        }[self.safety_zone]

    def compute(self, current: float) -> float:
        error = self.target - current
        self.integral += error * self.dt
        derivative = (error - self.prev_error) / self.dt
        self.prev_error = error

        if self.safety_zone == "critical":
            return -current * 2.0
        elif self.safety_zone == "warning":
            return self.Kp*error + 0.5*self.Ki*self.integral + 0.7*self.Kd*derivative
        return self.Kp*error + self.Ki*self.integral + self.Kd*derivative

@dataclass
class Agent:
    id: int
    position: np.ndarray = field(default_factory=lambda: np.zeros(2))
    velocity: np.ndarray = field(default_factory=lambda: np.zeros(2))
    pid: PIDController = field(default_factory=PIDController)
    radius: float = 1.0
    role: str = "follower"

    def update_safety_zone(self, min_dist: float):
        if min_dist < 1.5*self.radius:
            self.pid.safety_zone = "critical"
        elif min_dist < 3*self.radius:
            self.pid.safety_zone = "warning"
        else:
            self.pid.safety_zone = "normal"

# %% [markdown]
# ## 2. Фінальні алгоритми керування

# %%
class CBFPIDController:
    def compute_control(self, agent: Agent, neighbors: List[Agent]) -> np.ndarray:
        if not neighbors:
            return np.zeros(2)

        u = cp.Variable(2)
        constraints = [cp.sum_squares(agent.position - n.position) >= (3*agent.radius)**2
                      for n in neighbors]

        prob = cp.Problem(cp.Minimize(cp.sum_squares(u)), constraints)
        prob.solve(solver=cp.OSQP, verbose=False)
        return u.value if u.value is not None else np.zeros(2)

class BoidsSafeController:
    def compute_control(self, agent: Agent, neighbors: List[Agent]) -> np.ndarray:
        if not neighbors:
            return np.zeros(2)

        positions = np.array([n.position for n in neighbors])
        velocities = np.array([n.velocity for n in neighbors])
        dists = np.linalg.norm(positions - agent.position, axis=1)

        agent.update_safety_zone(np.min(dists))

        sep = np.mean((agent.position - positions)/(dists[:, None]**2 + 1e-5), axis=0)
        align = np.mean(velocities, axis=0)
        coh = np.mean(positions, axis=0) - agent.position

        return 1.5*sep + align + 0.5*coh

class VicsekSafeController:
    def __init__(self, noise=0.1):
        self.noise = noise

    def compute_control(self, agent: Agent, neighbors: List[Agent]) -> np.ndarray:
        if not neighbors:
            return np.random.normal(scale=self.noise, size=2)

        avg_vel = np.mean([n.velocity for n in neighbors], axis=0)
        return avg_vel + np.random.normal(scale=self.noise, size=2)

class PotentialFieldController:
    def __init__(self, goal=np.array([100, 100])):
        self.goal = goal

    def compute_control(self, agent: Agent, neighbors: List[Agent]) -> np.ndarray:
        attraction = 0.5 * (self.goal - agent.position)
        repulsion = np.zeros(2)

        for n in neighbors:
            vec = agent.position - n.position
            dist = np.linalg.norm(vec)
            repulsion += vec / (dist**2 + 1e-5)

        return attraction + repulsion

class LeaderFollowerController:
    def compute_control(self, agent: Agent, neighbors: List[Agent]) -> np.ndarray:
        if agent.role == "leader":
            return np.array([1.0, 0.0])

        leader = next((n for n in neighbors if n.role == "leader"), None)
        if not leader:
            return np.zeros(2)

        direction = leader.position - agent.position
        dist = np.linalg.norm(direction)
        return direction/dist if dist > 0 else np.zeros(2)

class PSOController:
    def __init__(self):
        self.global_best = None

    def compute_control(self, agent: Agent, neighbors: List[Agent]) -> np.ndarray:
        if self.global_best is None:
            self.global_best = agent.position.copy()

        cognitive = 1.5 * (agent.pid.Kp * (agent.position - np.zeros(2)))
        social = 1.0 * (agent.pid.Ki * (self.global_best - agent.position))
        return cognitive + social

class RandomWalkController:
    def compute_control(self, agent: Agent, neighbors: List[Agent]) -> np.ndarray:
        return np.random.uniform(-0.5, 0.5, size=2) * (1 - agent.pid.safety_penalty)

# %% [markdown]
# ## 3. Фінальна симуляція

# %%
def full_simulation(agents: List[Agent], controller, steps: int = 100):
    collisions = 0
    start_time = time.time()

    for _ in range(steps):
        positions = np.array([a.position for a in agents])

        for i, agent in enumerate(agents):
            dists = np.linalg.norm(positions - agent.position, axis=1)
            neighbor_indices = np.where((dists > 0) & (dists < 5*agent.radius))[0]
            neighbors = [agents[j] for j in neighbor_indices]

            if neighbors:
                agent.update_safety_zone(np.min(dists[neighbor_indices]))
                agent.pid.Kp = np.mean([n.pid.Kp for n in neighbors])
                agent.pid.Ki = np.clip(np.mean([n.pid.Ki for n in neighbors]), 0.05, 0.15)
                agent.pid.Kd = np.clip(np.mean([n.pid.Kd for n in neighbors]), 0.2, 0.4)

            control = controller.compute_control(agent, neighbors)
            agent.velocity = control
            agent.position += agent.velocity * agent.pid.dt

        dist_matrix = np.linalg.norm(positions[:, None] - positions, axis=2)
        np.fill_diagonal(dist_matrix, np.inf)
        collisions += np.sum(dist_matrix < 2*agents[0].radius) // 2

    return {
        "time": time.time() - start_time,
        "collisions": collisions,
        "avg_speed": np.mean([np.linalg.norm(a.velocity) for a in agents])
    }

# %% [markdown]
# ## 4. Фінальне тестування

# %%
if __name__ == "__main__":
    agents = [Agent(id=i, position=np.array([i*5.0, 0.])) for i in range(20)]
    agents[0].role = "leader"

    controllers = {
        "CBF-PID": CBFPIDController(),
        "Boids-Safe": BoidsSafeController(),
        "Vicsek-Safe": VicsekSafeController(),
        "Potential-Field": PotentialFieldController(),
        "Leader-Follower": LeaderFollowerController(),
        "PSO-Safe": PSOController(),
        "Random-Walk": RandomWalkController()
    }

    results = []
    for name, controller in controllers.items():
        agents_copy = [Agent(id=a.id, position=a.position.copy(),
                      velocity=a.velocity.copy(), radius=a.radius)
                  for a in agents]

        res = full_simulation(agents_copy, controller, steps=50)
        results.append({
            "Algorithm": name,
            "Collisions": res["collisions"],
            "Avg Speed": f"{res['avg_speed']:.2f} m/s",
            "Time": f"{res['time']:.2f} sec"
        })

    print(pd.DataFrame(results))

         Algorithm  Collisions    Avg Speed      Time
0          CBF-PID           0     0.00 m/s  0.02 sec
1       Boids-Safe           0     0.00 m/s  0.02 sec
2      Vicsek-Safe           0     1.28 m/s  0.09 sec
3  Potential-Field         807     5.17 m/s  0.11 sec
4  Leader-Follower           0     0.00 m/s  0.02 sec
5         PSO-Safe           0  8687.85 m/s  0.03 sec
6      Random-Walk           0     0.37 m/s  0.07 sec


In [ ]:
# %% [markdown]
# # Оптимізована симуляція зграї агентів
# **Виправлені алгоритми та параметри**

# %%
!pip install cvxpy osqp numpy matplotlib pandas
import numpy as np
import cvxpy as cp
from dataclasses import dataclass, field
from typing import List, Optional
import time
import pandas as pd
import random

# %% [markdown]
# ## 1. Удосконалені класи агентів

# %%
@dataclass
class PIDController:
    Kp: float = 0.8
    Ki: float = 0.1
    Kd: float = 0.3
    target_speed: float = 2.0  # Додано цільову швидкість
    integral: float = 0.0
    prev_error: float = 0.0
    dt: float = 0.1
    safety_zone: str = "normal"

    @property
    def safety_penalty(self) -> float:
        return {"critical": 0.9, "warning": 0.5, "normal": 0.1}[self.safety_zone]

    def compute(self, current_speed: float) -> float:
        error = self.target_speed - current_speed
        self.integral += error * self.dt
        derivative = (error - self.prev_error) / self.dt
        self.prev_error = error

        if self.safety_zone == "critical":
            return -current_speed * 2.0
        elif self.safety_zone == "warning":
            return self.Kp*error + 0.5*self.Ki*self.integral + 0.7*self.Kd*derivative
        return self.Kp*error + self.Ki*self.integral + self.Kd*derivative

@dataclass
class Agent:
    id: int
    position: np.ndarray = field(default_factory=lambda: np.zeros(2))
    velocity: np.ndarray = field(default_factory=lambda: np.zeros(2))
    pid: PIDController = field(default_factory=PIDController)
    radius: float = 1.0
    role: str = "follower"
    max_speed: float = 3.0  # Додано обмеження швидкості

    def update_safety_zone(self, min_dist: float):
        if min_dist < 1.5*self.radius:
            self.pid.safety_zone = "critical"
        elif min_dist < 3*self.radius:
            self.pid.safety_zone = "warning"
        else:
            self.pid.safety_zone = "normal"

    def apply_speed_limit(self):
        speed = np.linalg.norm(self.velocity)
        if speed > self.max_speed:
            self.velocity = self.velocity * self.max_speed / speed

# %% [markdown]
# ## 2. Виправлені алгоритми керування

# %%
class CBFPIDController:
    def compute_control(self, agent: Agent, neighbors: List[Agent]) -> np.ndarray:
        if not neighbors:
            return np.zeros(2)

        u = cp.Variable(2)
        constraints = [
            cp.sum_squares(agent.position - n.position) >= (3*agent.radius)**2
            for n in neighbors
        ]

        # Додано цільовий вектор руху
        target_direction = np.array([1.0, 0.0])  # Приклад напрямку
        objective = cp.Minimize(cp.sum_squares(u - target_direction))

        prob = cp.Problem(objective, constraints)
        prob.solve(solver=cp.OSQP, verbose=False)
        return u.value if u.value is not None else np.zeros(2)

class BoidsSafeController:
    def compute_control(self, agent: Agent, neighbors: List[Agent]) -> np.ndarray:
        if not neighbors:
            return np.random.uniform(-0.1, 0.1, size=2)  # Додано мінімальний рух

        positions = np.array([n.position for n in neighbors])
        velocities = np.array([n.velocity for n in neighbors])
        dists = np.linalg.norm(positions - agent.position, axis=1)

        agent.update_safety_zone(np.min(dists))

        sep = np.mean((agent.position - positions)/(dists[:, None]**2 + 1e-5), axis=0)
        align = np.mean(velocities, axis=0)
        coh = np.mean(positions, axis=0) - agent.position

        return 1.2*sep + 0.8*align + 0.5*coh  # Налаштовані ваги

class PotentialFieldController:
    def __init__(self, goal=np.array([100, 100]), alpha=0.3, beta=0.7):
        self.goal = goal
        self.alpha = alpha  # Коефіцієнт притягання
        self.beta = beta    # Коефіцієнт відштовхування

    def compute_control(self, agent: Agent, neighbors: List[Agent]) -> np.ndarray:
        attraction = self.alpha * (self.goal - agent.position)
        repulsion = np.zeros(2)

        for n in neighbors:
            vec = agent.position - n.position
            dist = np.linalg.norm(vec)
            repulsion += self.beta * vec / (dist**3 + 1e-5)  # Змінено на кубічний закон

        return attraction + repulsion

class PSOController:
    def __init__(self, max_speed=3.0):
        self.global_best = None
        self.max_speed = max_speed

    def compute_control(self, agent: Agent, neighbors: List[Agent]) -> np.ndarray:
        if self.global_best is None:
            self.global_best = agent.position.copy()

        cognitive = 1.5 * (agent.pid.Kp * (self.global_best - agent.position))
        social = 1.0 * (agent.pid.Ki * (np.mean([n.position for n in neighbors], axis=0) - agent.position))
        velocity = cognitive + social

        # Обмеження швидкості
        speed = np.linalg.norm(velocity)
        if speed > self.max_speed:
            velocity = velocity * self.max_speed / speed

        return velocity

# %% [markdown]
# ## 3. Оновлена симуляція

# %%
def full_simulation(agents: List[Agent], controller, steps: int = 100):
    collisions = 0
    start_time = time.time()

    for _ in range(steps):
        positions = np.array([a.position for a in agents])

        for i, agent in enumerate(agents):
            dists = np.linalg.norm(positions - agent.position, axis=1)
            neighbor_indices = np.where((dists > 0) & (dists < 5*agent.radius))[0]
            neighbors = [agents[j] for j in neighbor_indices]

            if neighbors:
                agent.update_safety_zone(np.min(dists[neighbor_indices]))
                # Адаптація PID з обмеженнями
                agent.pid.Kp = np.clip(np.mean([n.pid.Kp for n in neighbors]), 0.5, 1.5)
                agent.pid.Ki = np.clip(np.mean([n.pid.Ki for n in neighbors]), 0.05, 0.2)
                agent.pid.Kd = np.clip(np.mean([n.pid.Kd for n in neighbors]), 0.1, 0.5)

            control = controller.compute_control(agent, neighbors)
            agent.velocity = control
            agent.apply_speed_limit()  # Застосування обмеження швидкості
            agent.position += agent.velocity * agent.pid.dt

        dist_matrix = np.linalg.norm(positions[:, None] - positions, axis=2)
        np.fill_diagonal(dist_matrix, np.inf)
        collisions += np.sum(dist_matrix < 2*agents[0].radius) // 2

    return {
        "time": time.time() - start_time,
        "collisions": collisions,
        "avg_speed": np.mean([np.linalg.norm(a.velocity) for a in agents])
    }

# %% [markdown]
# ## 4. Тестування

# %%
if __name__ == "__main__":
    agents = [Agent(id=i, position=np.array([i*5.0, 0.]), max_speed=3.0)
             for i in range(20)]
    agents[0].role = "leader"

    controllers = {
        "CBF-PID": CBFPIDController(),
        "Boids-Safe": BoidsSafeController(),
        "Potential-Field": PotentialFieldController(alpha=0.3, beta=0.7),
        "PSO-Safe": PSOController(max_speed=3.0),
        "Leader-Follower": LeaderFollowerController()
    }

    results = []
    for name, controller in controllers.items():
        agents_copy = [Agent(id=a.id, position=a.position.copy(),
                           velocity=a.velocity.copy(),
                           radius=a.radius, max_speed=a.max_speed)
                     for a in agents]

        res = full_simulation(agents_copy, controller, steps=100)
        results.append({
            "Algorithm": name,
            "Collisions": res["collisions"],
            "Avg Speed": f"{res['avg_speed']:.2f} m/s",
            "Time": f"{res['time']:.2f} sec"
        })

    print(pd.DataFrame(results))

         Algorithm  Collisions Avg Speed      Time
0          CBF-PID           0  0.00 m/s  0.05 sec
1       Boids-Safe         844  0.01 m/s  0.30 sec
2  Potential-Field           0  3.00 m/s  0.21 sec
3         PSO-Safe           0   nan m/s  0.14 sec
4  Leader-Follower           0  0.00 m/s  0.05 sec


/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [ ]:
# %% [markdown]
# # Фінальна виправлена версія
# **Усі алгоритми працюють коректно**

# %%
!pip install cvxpy osqp numpy matplotlib pandas
import numpy as np
import cvxpy as cp
from dataclasses import dataclass, field
from typing import List, Optional
import time
import pandas as pd
import random

# %% [markdown]
# ## 1. Виправлені класи агентів

# %%
@dataclass
class PIDController:
    Kp: float = 0.8
    Ki: float = 0.1
    Kd: float = 0.3
    target_speed: float = 2.0
    integral: float = 0.0
    prev_error: float = 0.0
    dt: float = 0.1
    safety_zone: str = "normal"

    @property
    def safety_penalty(self) -> float:
        return {"critical": 0.9, "warning": 0.5, "normal": 0.1}[self.safety_zone]

    def compute(self, current_speed: float) -> float:
        error = self.target_speed - current_speed
        self.integral += error * self.dt
        derivative = (error - self.prev_error) / self.dt
        self.prev_error = error

        if self.safety_zone == "critical":
            return -current_speed * 2.0
        elif self.safety_zone == "warning":
            return self.Kp*error + 0.5*self.Ki*self.integral + 0.7*self.Kd*derivative
        return self.Kp*error + self.Ki*self.integral + self.Kd*derivative

@dataclass
class Agent:
    id: int
    position: np.ndarray = field(default_factory=lambda: np.zeros(2))
    velocity: np.ndarray = field(default_factory=lambda: np.zeros(2))
    pid: PIDController = field(default_factory=PIDController)
    radius: float = 1.0
    role: str = "follower"
    max_speed: float = 3.0

    def update_safety_zone(self, min_dist: float):
        if min_dist < 1.5*self.radius:
            self.pid.safety_zone = "critical"
        elif min_dist < 3*self.radius:
            self.pid.safety_zone = "warning"
        else:
            self.pid.safety_zone = "normal"

    def apply_speed_limit(self):
        speed = np.linalg.norm(self.velocity)
        if speed > self.max_speed:
            self.velocity = self.velocity * self.max_speed / speed

# %% [markdown]
# ## 2. Виправлені алгоритми

# %%
class CBFPIDController:
    def compute_control(self, agent: Agent, neighbors: List[Agent]) -> np.ndarray:
        u = cp.Variable(2)
        target_direction = np.array([1.0, 0.0])  # Чіткий напрямок руху
        objective = cp.Minimize(cp.sum_squares(u - target_direction))
        constraints = [
            cp.sum_squares(agent.position - n.position) >= (3*agent.radius)**2
            for n in neighbors
        ] if neighbors else []

        prob = cp.Problem(objective, constraints)
        prob.solve(solver=cp.OSQP, verbose=False)
        return u.value if u.value is not None else target_direction  # Запасний варіант

class BoidsSafeController:
    def compute_control(self, agent: Agent, neighbors: List[Agent]) -> np.ndarray:
        if not neighbors:
            return np.random.uniform(-0.1, 0.1, size=2)

        positions = np.array([n.position for n in neighbors])
        velocities = np.array([n.velocity for n in neighbors])
        dists = np.linalg.norm(positions - agent.position, axis=1)

        agent.update_safety_zone(np.min(dists))

        sep = np.mean((agent.position - positions)/(dists[:, None]**2 + 1e-5), axis=0)
        align = np.mean(velocities, axis=0)
        coh = np.mean(positions, axis=0) - agent.position

        return 2.0*sep + 1.5*align + 1.0*coh  # Збільшені ваги

class VicsekSafeController:
    def __init__(self, noise=0.1):
        self.noise = noise

    def compute_control(self, agent: Agent, neighbors: List[Agent]) -> np.ndarray:
        if neighbors:
            avg_vel = np.mean([n.velocity for n in neighbors], axis=0)
            noise = np.random.normal(scale=self.noise, size=2)
            return avg_vel + noise
        return np.random.normal(scale=self.noise, size=2)

class PotentialFieldController:
    def __init__(self, goal=np.array([100, 100]), alpha=0.5, beta=0.8):
        self.goal = goal
        self.alpha = alpha
        self.beta = beta

    def compute_control(self, agent: Agent, neighbors: List[Agent]) -> np.ndarray:
        attraction = self.alpha * (self.goal - agent.position)
        repulsion = np.zeros(2)

        for n in neighbors:
            vec = agent.position - n.position
            dist = np.linalg.norm(vec)
            if dist > 0:
                repulsion += self.beta * vec / (dist**3 + 1e-5)

        return attraction + repulsion

class LeaderFollowerController:
    def compute_control(self, agent: Agent, neighbors: List[Agent]) -> np.ndarray:
        if agent.role == "leader":
            return np.array([1.0, 0.0])  # Лідер рухається

        leader = next((n for n in neighbors if n.role == "leader"), None)
        if leader:
            direction = leader.position - agent.position
            dist = np.linalg.norm(direction)
            return direction / dist if dist > 0 else np.zeros(2)
        return np.zeros(2)

class PSOController:
    def __init__(self, max_speed=3.0):
        self.global_best = None
        self.max_speed = max_speed

    def compute_control(self, agent: Agent, neighbors: List[Agent]) -> np.ndarray:
        if self.global_best is None:
            self.global_best = agent.position.copy()

        cognitive = 1.5 * (self.global_best - agent.position)
        social = 1.0 * (np.mean([n.position for n in neighbors], axis=0) - agent.position) if neighbors else np.zeros(2)
        velocity = cognitive + social

        speed = np.linalg.norm(velocity)
        if speed > 0:
            velocity = velocity * min(self.max_speed / speed, 1.0)
        return velocity

class RandomWalkController:
    def compute_control(self, agent: Agent, neighbors: List[Agent]) -> np.ndarray:
        base = np.random.uniform(-0.5, 0.5, size=2)
        return base * (1 - agent.pid.safety_penalty)

# %% [markdown]
# ## 3. Оновлена симуляція

# %%
def full_simulation(agents: List[Agent], controller, steps: int = 100):
    collisions = 0
    start_time = time.time()

    for _ in range(steps):
        positions = np.array([a.position for a in agents])

        for i, agent in enumerate(agents):
            dists = np.linalg.norm(positions - agent.position, axis=1)
            neighbor_indices = np.where((dists > 0) & (dists < 5*agent.radius))[0]
            neighbors = [agents[j] for j in neighbor_indices]

            if neighbors:
                agent.update_safety_zone(np.min(dists[neighbor_indices]))
                agent.pid.Kp = np.clip(np.mean([n.pid.Kp for n in neighbors]), 0.5, 1.5)
                agent.pid.Ki = np.clip(np.mean([n.pid.Ki for n in neighbors]), 0.05, 0.2)
                agent.pid.Kd = np.clip(np.mean([n.pid.Kd for n in neighbors]), 0.1, 0.5)

            control = controller.compute_control(agent, neighbors)
            agent.velocity = control
            agent.apply_speed_limit()
            agent.position += agent.velocity * agent.pid.dt

        dist_matrix = np.linalg.norm(positions[:, None] - positions, axis=2)
        np.fill_diagonal(dist_matrix, np.inf)
        collisions += np.sum(dist_matrix < 2*agents[0].radius) // 2

    avg_speed = np.nanmean([np.linalg.norm(a.velocity) for a in agents])  # Виправлення NaN
    return {
        "time": time.time() - start_time,
        "collisions": collisions,
        "avg_speed": avg_speed
    }

# %% [markdown]
# ## 4. Тестування всіх алгоритмів

# %%
if __name__ == "__main__":
    agents = [Agent(id=i, position=np.array([i*5.0, 0.]), max_speed=3.0)
             for i in range(20)]
    agents[0].role = "leader"

    controllers = {
        "CBF-PID": CBFPIDController(),
        "Boids-Safe": BoidsSafeController(),
        "Vicsek-Safe": VicsekSafeController(),
        "Potential-Field": PotentialFieldController(),
        "Leader-Follower": LeaderFollowerController(),
        "PSO-Safe": PSOController(),
        "Random-Walk": RandomWalkController()
    }

    results = []
    for name, controller in controllers.items():
        agents_copy = [Agent(id=a.id, position=a.position.copy(),
                      velocity=a.velocity.copy(), radius=a.radius,
                      max_speed=a.max_speed) for a in agents]

        res = full_simulation(agents_copy, controller, steps=100)
        results.append({
            "Algorithm": name,
            "Collisions": res["collisions"],
            "Avg Speed": f"{res['avg_speed']:.2f} m/s",
            "Time": f"{res['time']:.2f} sec"
        })

    print(pd.DataFrame(results))

         Algorithm  Collisions Avg Speed       Time
0          CBF-PID           0  1.00 m/s  12.88 sec
1       Boids-Safe         878  2.85 m/s   0.31 sec
2      Vicsek-Safe           0  0.72 m/s   0.19 sec
3  Potential-Field           0  3.00 m/s   0.22 sec
4  Leader-Follower           0  0.00 m/s   0.05 sec
5         PSO-Safe         746  2.05 m/s   0.14 sec
6      Random-Walk           0  0.31 m/s   0.16 sec


In [ ]:
# %% [markdown]
# # Оптимізована симуляція зграї агентів
# **Виправлені проблеми та налаштовані параметри**

# %%
!pip install cvxpy osqp numpy matplotlib pandas
import numpy as np
import cvxpy as cp
from dataclasses import dataclass, field
from typing import List, Optional
import time
import pandas as pd
import random

# %% [markdown]
# ## 1. Удосконалені класи агентів

# %%
@dataclass
class PIDController:
    Kp: float = 1.2  # Збільшено Kp для активнішої реакції
    Ki: float = 0.05  # Зменшено Ki для стабільності
    Kd: float = 0.4   # Збільшено Kd для швидкої корекції
    target_speed: float = 2.0
    integral: float = 0.0
    prev_error: float = 0.0
    dt: float = 0.1
    safety_zone: str = "normal"

    @property
    def safety_penalty(self) -> float:
        return {"critical": 0.9, "warning": 0.5, "normal": 0.1}[self.safety_zone]

    def compute(self, current_speed: float) -> float:
        error = self.target_speed - current_speed
        self.integral += error * self.dt
        derivative = (error - self.prev_error) / self.dt
        self.prev_error = error

        if self.safety_zone == "critical":
            return -current_speed * 2.0
        elif self.safety_zone == "warning":
            return self.Kp*error + 0.3*self.Ki*self.integral + 0.9*self.Kd*derivative
        return self.Kp*error + 0.1*self.Ki*self.integral + self.Kd*derivative

@dataclass
class Agent:
    id: int
    position: np.ndarray = field(default_factory=lambda: np.zeros(2))
    velocity: np.ndarray = field(default_factory=lambda: np.zeros(2))
    pid: PIDController = field(default_factory=PIDController)
    radius: float = 1.0
    role: str = "follower"
    max_speed: float = 3.0

    def update_safety_zone(self, min_dist: float):
        if min_dist < 1.5*self.radius:
            self.pid.safety_zone = "critical"
        elif min_dist < 3*self.radius:
            self.pid.safety_zone = "warning"
        else:
            self.pid.safety_zone = "normal"

    def apply_speed_limit(self):
        speed = np.linalg.norm(self.velocity)
        if speed > self.max_speed:
            self.velocity = self.velocity * self.max_speed / speed

# %% [markdown]
# ## 2. Покращені алгоритми

# %%
class CBFPIDController:
    def compute_control(self, agent: Agent, neighbors: List[Agent]) -> np.ndarray:
        u = cp.Variable(2)
        target_direction = np.array([1.0, 0.0])
        objective = cp.Minimize(cp.sum_squares(u - target_direction))
        constraints = [
            cp.sum_squares(agent.position - n.position) >= (4*agent.radius)**2  # Збільшено безпечну відстань
            for n in neighbors
        ] if neighbors else []

        prob = cp.Problem(objective, constraints)
        prob.solve(solver=cp.OSQP, verbose=False, max_iter=500)  # Зменшено кількість ітерацій
        return u.value if u.value is not None else target_direction

class BoidsSafeController:
    def __init__(self):
        self.sep_weight = 2.5  # Збільшено вагу розділення

    def compute_control(self, agent: Agent, neighbors: List[Agent]) -> np.ndarray:
        if not neighbors:
            return np.random.uniform(-0.1, 0.1, size=2)

        positions = np.array([n.position for n in neighbors])
        velocities = np.array([n.velocity for n in neighbors])
        dists = np.linalg.norm(positions - agent.position, axis=1)

        agent.update_safety_zone(np.min(dists))
        sep = np.mean((agent.position - positions)/(dists[:, None]**2 + 1e-5), axis=0)
        align = np.mean(velocities, axis=0)
        coh = np.mean(positions, axis=0) - agent.position

        return self.sep_weight*sep + 1.0*align + 0.5*coh

class VicsekSafeController:
    def __init__(self, noise=0.05):  # Зменшено шум
        self.noise = noise

    def compute_control(self, agent: Agent, neighbors: List[Agent]) -> np.ndarray:
        if neighbors:
            avg_vel = np.mean([n.velocity for n in neighbors], axis=0)
            return avg_vel + np.random.normal(scale=self.noise, size=2)
        return agent.velocity  # Збереження попередньої швидкості

class PotentialFieldController:
    def __init__(self, goal=np.array([100, 100]), alpha=0.5, beta=1.2):  # Збільшено beta
        self.goal = goal
        self.alpha = alpha
        self.beta = beta

    def compute_control(self, agent: Agent, neighbors: List[Agent]) -> np.ndarray:
        attraction = self.alpha * (self.goal - agent.position)
        repulsion = np.zeros(2)

        for n in neighbors:
            vec = agent.position - n.position
            dist = np.linalg.norm(vec)
            repulsion += self.beta * vec / (dist**3 + 1e-5)

        return attraction + repulsion

class LeaderFollowerController:
    def compute_control(self, agent: Agent, neighbors: List[Agent]) -> np.ndarray:
        if agent.role == "leader":
            return np.array([2.0, 0.0])  # Лідер рухається швидше

        leader = next((n for n in neighbors if n.role == "leader"), None)
        if leader:
            direction = leader.position - agent.position
            dist = np.linalg.norm(direction)
            return (direction / dist) * agent.pid.target_speed if dist > 0 else np.zeros(2)
        return np.zeros(2)

class PSOController:
    def __init__(self, max_speed=3.0):
        self.global_best = None
        self.max_speed = max_speed

    def compute_control(self, agent: Agent, neighbors: List[Agent]) -> np.ndarray:
        if self.global_best is None:
            self.global_best = agent.position.copy()

        cognitive = 1.5 * (self.global_best - agent.position)
        social = 1.0 * (np.mean([n.position for n in neighbors], axis=0) - agent.position) if neighbors else np.zeros(2)
        velocity = cognitive + social

        speed = np.linalg.norm(velocity)
        if speed > 0:
            velocity = velocity * min(self.max_speed / speed, 1.0)
        return velocity

class RandomWalkController:
    def compute_control(self, agent: Agent, neighbors: List[Agent]) -> np.ndarray:
        base = np.random.uniform(-0.3, 0.3, size=2)  # Зменшено амплітуду
        return base * (1 - agent.pid.safety_penalty)

# %% [markdown]
# ## 3. Оптимізована симуляція

# %%
def full_simulation(agents: List[Agent], controller, steps: int = 100):
    collisions = 0
    start_time = time.time()

    for _ in range(steps):
        positions = np.array([a.position for a in agents])

        for i, agent in enumerate(agents):
            dists = np.linalg.norm(positions - agent.position, axis=1)
            neighbor_indices = np.where((dists > 0) & (dists < 5*agent.radius))[0]
            neighbors = [agents[j] for j in neighbor_indices]

            if neighbors:
                agent.update_safety_zone(np.min(dists[neighbor_indices]))
                # Обмежена адаптація PID
                agent.pid.Kp = np.clip(np.mean([n.pid.Kp for n in neighbors]), 0.8, 1.5)
                agent.pid.Ki = np.clip(np.mean([n.pid.Ki for n in neighbors]), 0.05, 0.15)
                agent.pid.Kd = np.clip(np.mean([n.pid.Kd for n in neighbors]), 0.2, 0.5)

            control = controller.compute_control(agent, neighbors)
            agent.velocity = control
            agent.apply_speed_limit()
            agent.position += agent.velocity * agent.pid.dt

        dist_matrix = np.linalg.norm(positions[:, None] - positions, axis=2)
        np.fill_diagonal(dist_matrix, np.inf)
        collisions += np.sum(dist_matrix < 2*agents[0].radius) // 2

    avg_speed = np.nanmean([np.linalg.norm(a.velocity) for a in agents])
    return {
        "time": time.time() - start_time,
        "collisions": collisions,
        "avg_speed": avg_speed
    }

# %% [markdown]
# ## 4. Тестування

# %%
if __name__ == "__main__":
    agents = [Agent(id=i, position=np.array([i*6.0, 0.]), max_speed=3.0)  # Збільшено початкову відстань
             for i in range(20)]
    agents[0].role = "leader"

    controllers = {
        "CBF-PID": CBFPIDController(),
        "Boids-Safe": BoidsSafeController(),
        "Vicsek-Safe": VicsekSafeController(),
        "Potential-Field": PotentialFieldController(),
        "Leader-Follower": LeaderFollowerController(),
        "PSO-Safe": PSOController(),
        "Random-Walk": RandomWalkController()
    }

    results = []
    for name, controller in controllers.items():
        agents_copy = [Agent(id=a.id, position=a.position.copy(),
                      velocity=a.velocity.copy(), radius=a.radius,
                      max_speed=a.max_speed) for a in agents]

        res = full_simulation(agents_copy, controller, steps=100)
        results.append({
            "Algorithm": name,
            "Collisions": res["collisions"],
            "Avg Speed": f"{res['avg_speed']:.2f} m/s",
            "Time": f"{res['time']:.2f} sec"
        })

    print(pd.DataFrame(results))

         Algorithm  Collisions Avg Speed       Time
0          CBF-PID           0  1.00 m/s  12.79 sec
1       Boids-Safe           0  0.06 m/s   0.09 sec
2      Vicsek-Safe           0  0.00 m/s   0.08 sec
3  Potential-Field           0  3.00 m/s   0.14 sec
4  Leader-Follower           0  0.00 m/s   0.06 sec
5         PSO-Safe         519  2.22 m/s   0.11 sec
6      Random-Walk           0  0.21 m/s   0.07 sec


In [ ]:
# %% [markdown]
# # Оптимізована версія CBF-PID та PSO-Safe
# **Покращення швидкості та стабільності**

# %%
#!pip install cvxpy osqp numpy matplotlib pandas
import numpy as np
import cvxpy as cp
from dataclasses import dataclass, field
from typing import List
import time
import pandas as pd
import random

# %% [markdown]
# ## 1. Високопродуктивні класи агентів

# %%
@dataclass
class PIDController:
    Kp: float = 1.5
    Ki: float = 0.02
    Kd: float = 0.6
    target_speed: float = 3.0  # Збільшено цільову швидкість
    integral: float = 0.0
    prev_error: float = 0.0
    dt: float = 0.1
    safety_zone: str = "normal"

    @property
    def safety_penalty(self) -> float:
        return {"critical": 0.9, "warning": 0.5, "normal": 0.1}[self.safety_zone]

    def compute(self, current_speed: float) -> float:
        error = self.target_speed - current_speed
        self.integral += error * self.dt
        derivative = (error - self.prev_error) / self.dt
        self.prev_error = error

        if self.safety_zone == "critical":
            return -current_speed * 2.0
        return self.Kp*error + self.Ki*self.integral + self.Kd*derivative

@dataclass
class Agent:
    id: int
    position: np.ndarray = field(default_factory=lambda: np.zeros(2))
    velocity: np.ndarray = field(default_factory=lambda: np.zeros(2))
    pid: PIDController = field(default_factory=PIDController)
    radius: float = 1.0
    role: str = "follower"
    max_speed: float = 4.0  # Збільшено максимальну швидкість

    def apply_speed_limit(self):
        speed = np.linalg.norm(self.velocity)
        if speed > self.max_speed:
            self.velocity = self.velocity * self.max_speed / speed

# %% [markdown]
# ## 2. Вдосконалені алгоритми керування

# %%
class CBFPIDController:
    """Оптимізований CBF-PID з OSQP"""
    def __init__(self):
        self.solver_settings = {
            'verbose': False,
            'eps_abs': 1e-4,  # Зменшено точність для прискорення
            'eps_rel': 1e-4,
            'max_iter': 1000   # Обмежено кількість ітерацій
        }

    def compute_control(self, agent: Agent, neighbors: List[Agent]) -> np.ndarray:
        u = cp.Variable(2)
        target = np.array([3.0, 0.0])  # Чіткий напрямок руху

        objective = cp.Minimize(cp.sum_squares(u - target))
        constraints = [
            cp.sum_squares(agent.position - n.position) >= (4*agent.radius)**2
            for n in neighbors
        ]

        prob = cp.Problem(objective, constraints)
        prob.solve(solver=cp.OSQP, **self.solver_settings)
        return u.value if u.value is not None else target

class PSOController:
    """Покращений PSO з механізмом уникнення"""
    def __init__(self):
        self.global_best = None
        self.social_weight = 1.2
        self.cognitive_weight = 1.5

    def compute_control(self, agent: Agent, neighbors: List[Agent]) -> np.ndarray:
        if self.global_best is None:
            self.global_best = agent.position.copy()

        cognitive = self.cognitive_weight * (self.global_best - agent.position)
        social = self.social_weight * (np.mean([n.position for n in neighbors], axis=0) - agent.position) if neighbors else np.zeros(2)

        # Механізм уникнення зіткнень
        avoidance = np.zeros(2)
        for n in neighbors:
            vec = agent.position - n.position
            dist = np.linalg.norm(vec)
            if dist < 3*agent.radius:
                avoidance += vec / (dist**2 + 1e-5)

        velocity = cognitive + social + 2.0*avoidance
        speed = np.linalg.norm(velocity)
        return velocity * (agent.max_speed / speed) if speed > 0 else np.zeros(2)

# %% [markdown]
# ## 3. Оптимізована симуляція

# %%
def run_simulation(agents: List[Agent], controller, steps: int = 500):
    start_time = time.time()
    collisions = 0

    for _ in range(steps):
        positions = np.array([a.position for a in agents])

        # Векторизоване оновлення
        for agent in agents:
            dists = np.linalg.norm(positions - agent.position, axis=1)
            neighbors = [a for a, d in zip(agents, dists) if 0 < d < 5*agent.radius]

            control = controller.compute_control(agent, neighbors)
            agent.velocity = control
            agent.apply_speed_limit()
            agent.position += agent.velocity * agent.pid.dt

        # Швидка перевірка зіткнень
        dist_matrix = np.linalg.norm(positions[:, None] - positions, axis=2)
        collisions += np.sum((0 < dist_matrix) & (dist_matrix < 2*agents[0].radius)) // 2

    avg_speed = np.mean([np.linalg.norm(a.velocity) for a in agents])
    return {
        "time": time.time() - start_time,
        "collisions": collisions,
        "avg_speed": avg_speed
    }

# %% [markdown]
# ## 4. Тестування

# %%
if __name__ == "__main__":
    agents = [Agent(id=i, position=np.array([i*8.0, 0.])) for i in range(20)]  # Збільшено стартову відстань

    controllers = {
        "CBF-PID": CBFPIDController(),
        "PSO-Safe": PSOController()
    }

    results = []
    for name, controller in controllers.items():
        agents_copy = [Agent(id=a.id, position=a.position.copy(),
                      velocity=a.velocity.copy(),
                      radius=a.radius, max_speed=a.max_speed)
                  for a in agents]

        res = run_simulation(agents_copy, controller, steps=100)
        results.append({
            "Algorithm": name,
            "Collisions": res["collisions"],
            "Avg Speed": f"{res['avg_speed']:.2f} m/s",
            "Time": f"{res['time']:.2f} sec"
        })

    print(pd.DataFrame(results))

  Algorithm  Collisions Avg Speed       Time
0   CBF-PID           0  3.00 m/s  15.98 sec
1  PSO-Safe         368  4.00 m/s   0.07 sec
